# Narrative for conducting a raffle with Marlowe

## Prerequisite

### How to create signing keys at the command line, using a seed phrase

Here is the recipe for create signing key files that can be used at the command line, starting with a seed phrase that can be used in a standard Cardano wallet. This lets one view and perform transactions both at the command line and in the wallet.

***Remember to use best security practices for protecting seed phrases and signing key files.*** To limit the attack surface, it is best to not reuse this wallet after the raffle, since the seed phrase or signing keys are stored on a computer connected to the internet during the raffle.

First start with a 24-word seed phrase either by generating one, as shown below, or by using one generated by a wallet.

In [1]:
cardano-wallet recovery-phrase generate | tee sponsor.seed

pet talent must join rigid radio kick airport illegal attend belt flip museum relax tragic april grit hurt because bonus wear physical episode any


Next derive the root private key from the seed in [sponsor.root.prv](sponsor.root.prv).

In [2]:
cat sponsor.seed \
| cardano-wallet key from-recovery-phrase Shelley \
> sponsor.root.prv

We derive the stake key from the root private key.

In [3]:
cat sponsor.root.prv \
| cardano-wallet key child 1852H/1815H/0H/2/0 \
| cardano-cli key convert-cardano-address-key \
    --shelley-stake-key \
    --signing-key-file /dev/stdin \
    --out-file /dev/stdout \
| cardano-cli key verification-key \
    --signing-key-file /dev/stdin \
    --verification-key-file /dev/stdout \
| cardano-cli key non-extended-key \
    --extended-verification-key-file /dev/stdin \
    --verification-key-file sponsor.stake.vkey

cardano-cli stake-address build \
  --mainnet \
  --stake-verification-key-file sponsor.stake.vkey \
| tee sponsor.stake.mainnet.address

stake1u92tacjl8h9alww05fjytcs2qucmgm79xwgrkwdckge705g4u5r52


A Shelley wallet can have multiple addresses, but we just derive the first address here. If one is using the same seed phrase in a web wallet, setting that wallet to single-address mode will ensure that the wallet sends change to the first address. Otherwise, the web wallet would spread UTxOs among multiple addresses, one the first of which is derived here.

For example, the Eternl wallet has a single-address mode configuration on its settings page.

![Single-address mode in the Eternl wallet](single-address-mode.png)

Now derive the wallet's first payment address.

In [4]:
cat sponsor.root.prv \
| cardano-wallet key child 1852H/1815H/0H/0/0 \
| cardano-cli key convert-cardano-address-key \
    --shelley-payment-key \
    --signing-key-file /dev/stdin \
    --out-file sponsor.skey

cardano-cli key verification-key \
  --signing-key-file sponsor.skey \
  --verification-key-file /dev/stdout \
| cardano-cli address build \
    --mainnet \
    --payment-verification-key-file /dev/stdin \
    --stake-verification-key-file sponsor.stake.vkey \
| tee sponsor.mainnet.address

addr1q8m2m2ph3y63ndlr0wt24lc9qts09vhxwcrpl2fggjpjwe25hm3970wtm7uulgnygh3q5pe3k3hu2vus8vum3v3nulgszmmrsm

So now we have the following:
- Seed phrase: [sponsor.seed](sponsor.seed)
- Root private key: [sponsor.root.prv](sponsor.root.prv)
- Stake address: [sponsor.mainnet.stake.address](sponsor.mainnet.stake.address)
- First payment signing key: [sponsor.skey](sponsor.skey)
- First payment address: [sponsor.mainnet.address](sponsor.mainnet.address)

The signing key file corresponds to the `1852H/1815H/0H/0/0` key derivation in the web wallet.

Note that the above procedure can be used for generating the corresponding testnet keys if `--mainnet` is replaced by `--testnet 1` and the `mainnet` in the filenames is replaced by `testnet`.

## Organizing the prizes

The ada and tokens for each prize must be organized into a separate UTxO at Marlowe's role-payout address and ***with the correct datum***.

Most web wallet cannot attach datum to a transaction output, so we have use the command line to build the transactions for bundling the prizes.

### Recording the value of the prizes

The ada for the prizes is:
- First prize: `3000 ada`
- Second prize: `500 ada`
- Third prize: `250 ada`

Each of the three prizes also include a set of native tokens. These tokens currently reside in three wallets:
- First prizes: [stake1u94pal56jgt47sppswkvd72m2swgdqp3gwg2dch6x6p8kkczakl8d](https://cardanoscan.io/stakeKey/stake1u94pal56jgt47sppswkvd72m2swgdqp3gwg2dch6x6p8kkczakl8d)
- Second prizes: [stake1u9h3ny9j9nmvzv9rnnvafvrafzcj94myu584u5w87snmm9gk8n3hu](https://cardanoscan.io/stakeKey/stake1u9h3ny9j9nmvzv9rnnvafvrafzcj94myu584u5w87snmm9gk8n3hu)
- Third prizes: [stake1uxj92pk8w6ufhv6zx83qww9plhy9q3kt863nwcrkucd9ppgay56v5](https://cardanoscan.io/stakeKey/stake1uxj92pk8w6ufhv6zx83qww9plhy9q3kt863nwcrkucd9ppgay56v5)

Currently, each wallet contains UTxOs at several addresses:

In [5]:
PRIZE1_ADDR=(
  addr1qyqx3gm0vlvpj8qpd6m8gk0nxrz329ngrycffygjdk4z0zt2rmlf4yshtaqzrqavcmu4k4qus6qrzsus5m305d5z0ddscyxjnl
  addr1q82w94tqv3ehc6zk3zt9wpc8cjq62p3sxsm8ewuzq9m64um2rmlf4yshtaqzrqavcmu4k4qus6qrzsus5m305d5z0ddskwts8w
  addr1q9jd3pt549enh0qydun34awjt9j36qj6shqem9tcwh3eazt2rmlf4yshtaqzrqavcmu4k4qus6qrzsus5m305d5z0dds6e652z
)
PRIZE2_ADDR=(
  addr1qx68rvkaclh4kkeae7px973ee508ve2jqv64ka78eh6yy8n0rxgtyt8kcyc288xe6jc86j93yttkfeg0tegu0ap8hk2s5k73zu
  addr1q80u7xl9ga8n8rgwm2wtm8jwnde040d7r3gq8rqfegt3tqt0rxgtyt8kcyc288xe6jc86j93yttkfeg0tegu0ap8hk2s6360ww
  addr1q9kexm3peu8dm2exyh8yqvpjthf3pw294sla49yuwuhekrr0rxgtyt8kcyc288xe6jc86j93yttkfeg0tegu0ap8hk2shwn25z
  addr1qy7qjjmpmmltfs2h78qykhsp9fjzt442v5ww57zfvurv45n0rxgtyt8kcyc288xe6jc86j93yttkfeg0tegu0ap8hk2sgk2wpe
  addr1q9htdsy7dnt0tcpypa830tsqzhms5kkl656mvxrj2een2kt0rxgtyt8kcyc288xe6jc86j93yttkfeg0tegu0ap8hk2s8cs0v9
)
PRIZE3_ADDR=(
  addr1q8cz6kchzgq7pvcf2970g3ru8e76v5kx7f4xtddpcvmyw84y25rvwa4cnwe5yv0zquu2rlwg2prvk04rxas8des62zzstgr226
)

We can query those addresses with `cardano-cli` to retrieve the value of ada and native tokens for the prizes:

In [6]:
export CARDANO_NODE_SOCKET_PATH=node.socket
ADA=1000000

In [7]:
PRIZE1_VALUE=$((3000 * ADA))+$(
cardano-cli query utxo \
  --mainnet \
  --address ${PRIZE1_ADDR[0]} \
  --address ${PRIZE1_ADDR[1]} \
  --address ${PRIZE1_ADDR[2]} \
| sed -e '1,2d' -e '/lovelace + TxOutDatumNone/d' -e 's/^.*lovelace + \(.*\) + TxOutDatumNone$/"\1"/' \
| jq -rs '. | join("+")' \
)
echo "PRIZE1_VALUE = $PRIZE1_VALUE"

PRIZE1_VALUE = 3000000000+1 d99c8d9126d8be2e35f07c8169290acc22889b0cdc9634efc2a3e068.4d6574657261426c75653236+125000 160a880d9fc45380737cb7e57ff859763230aab28b3ef6a84007bfcc.4d495241 + 1 1d779e04b2ee9d64acb9e747129c10760976ad651696d7e406c39acb.4d696c6573746f6e6531434e43416c6131353837 + 1 1d779e04b2ee9d64acb9e747129c10760976ad651696d7e406c39acb.4d696c6573746f6e6532434e43416c6131363238 + 1 1d779e04b2ee9d64acb9e747129c10760976ad651696d7e406c39acb.4d696c6573746f6e6533434e43416c6131323633 + 1 1d779e04b2ee9d64acb9e747129c10760976ad651696d7e406c39acb.4d696c6573746f6e6534434e43416c6130393030 + 1 1d779e04b2ee9d64acb9e747129c10760976ad651696d7e406c39acb.4d696c6573746f6e6536434e43416c6131353337 + 700000000 20cd68533b47565f3c61efb39c30fdace9963bfa4c0060b613448e3c.50524f584945 + 1 2a6c1ea5b4805b1aabae53758e9e2e6623e7332cded72fef7e1ff8bf.50726f786965576f6d62 + 1 43528549ec02054e09bcdadb34d27fcf18f699f1e925a86afd483066.000de1404f544b50697261746532343835 + 60000000000 682fe60c9918842b3323c43b5144bc3d5

In [8]:
PRIZE2_VALUE=$((500 * ADA))+$(
cardano-cli query utxo \
  --mainnet \
  --address ${PRIZE2_ADDR[0]} \
  --address ${PRIZE2_ADDR[1]} \
  --address ${PRIZE2_ADDR[2]} \
  --address ${PRIZE2_ADDR[3]} \
  --address ${PRIZE2_ADDR[4]} \
| sed -e '1,2d' -e '/lovelace + TxOutDatumNone/d' -e 's/^.*lovelace + \(.*\) + TxOutDatumNone$/"\1"/' \
| jq -rs '. | join("+")' \
)
echo "PRIZE2_VALUE = $PRIZE2_VALUE"

PRIZE2_VALUE = 500000000+1 f0ff48bbb7bbe9d59a40f1ce90e9e9d0ff5002ec48f232b49ca0fb9a.636e632e616c612d726166666c6532+50000 160a880d9fc45380737cb7e57ff859763230aab28b3ef6a84007bfcc.4d495241 + 1 1d779e04b2ee9d64acb9e747129c10760976ad651696d7e406c39acb.4d696c6573746f6e6531434e43416c6130363239 + 1 1d779e04b2ee9d64acb9e747129c10760976ad651696d7e406c39acb.4d696c6573746f6e6532434e43416c6131353338 + 1 1d779e04b2ee9d64acb9e747129c10760976ad651696d7e406c39acb.4d696c6573746f6e6533434e43416c6130373839 + 1 1d779e04b2ee9d64acb9e747129c10760976ad651696d7e406c39acb.4d696c6573746f6e6534434e43416c6130333535 + 1 1d779e04b2ee9d64acb9e747129c10760976ad651696d7e406c39acb.4d696c6573746f6e6535434e43416c6131343835 + 1 1d779e04b2ee9d64acb9e747129c10760976ad651696d7e406c39acb.4d696c6573746f6e6536434e43416c6131323732 + 1 1d779e04b2ee9d64acb9e747129c10760976ad651696d7e406c39acb.4d696c6573746f6e6537434e43416c6131303033 + 200000000 20cd68533b47565f3c61efb39c30fdace9963bfa4c0060b613448e3c.50524f584945 + 1 354da60572a38

In [9]:
PRIZE3_VALUE=$((250 * ADA))+$(
cardano-cli query utxo \
  --mainnet \
  --address ${PRIZE3_ADDR[0]} \
| sed -e '1,2d' -e '/lovelace + TxOutDatumNone/d' -e 's/^.*lovelace + \(.*\) + TxOutDatumNone$/"\1"/' \
| jq -rs '. | join("+")' \
)
echo "PRIZE3_VALUE = $PRIZE3_VALUE"

PRIZE3_VALUE = 250000000+1 2103673b9f5ee43408d78ad02805bb7bb8b2e118ffc9fb5ff81c28d3.4d65636861537461673032333331+100000000 c6de3e6ed46a935ffd7d4c560004845b5863f97e1d1806eeedc83654.50494e4b50414e54484552+1 f0ff48bbb7bbe9d59a40f1ce90e9e9d0ff5002ec48f232b49ca0fb9a.636e632e616c612d726166666c6533+25000 160a880d9fc45380737cb7e57ff859763230aab28b3ef6a84007bfcc.4d495241 + 1 1d779e04b2ee9d64acb9e747129c10760976ad651696d7e406c39acb.4d696c6573746f6e6531434e43416c6130303634 + 1 1d779e04b2ee9d64acb9e747129c10760976ad651696d7e406c39acb.4d696c6573746f6e6532434e43416c6131333239 + 1 1d779e04b2ee9d64acb9e747129c10760976ad651696d7e406c39acb.4d696c6573746f6e6533434e43416c6130373233 + 1 1d779e04b2ee9d64acb9e747129c10760976ad651696d7e406c39acb.4d696c6573746f6e6534434e43416c6130303434 + 1 1d779e04b2ee9d64acb9e747129c10760976ad651696d7e406c39acb.4d696c6573746f6e6535434e43416c6131333730 + 1 1d779e04b2ee9d64acb9e747129c10760976ad651696d7e406c39acb.4d696c6573746f6e6536434e43416c6130333635 + 1 1d779e04b2ee9d64acb

Note that the hexademical digits following each token's policy ID encode the token's name. For example, the `WisdomOrdinals0302` token is encoded as `576973646f6d4f7264696e616c7330333032`.

In [10]:
echo -n 576973646f6d4f7264696e616c7330333032 | tr '[:lower:]' '[:upper:]' | basenc --decode --base16

WisdomOrdinals0302

### Bundling the prizes at Marlowe's role-payout address

Because most web wallets cannot attach a datum hash to a transaction, we need to move the prizes to a staging are and do that at the command line.

Now we compute the Marlowe role-payout address.

In [11]:
PAYOUT_ADDR=$(marlowe-cli role address --mainnet)
echo "PAYOUT_ADDR = $PAYOUT_ADDR"

PAYOUT_ADDR = addr1w8sk2cgzxg34hwa7ladenzer8k4wg2temmyj5eezm8x6nzg35nqj2


#### First prize

We need to know the policy ID and token name for the NFT serves as the Marlowe role token for redeeming the first prize. There is no requirement for this token beyond that we know its policy ID and token name.

For the purposes of this narrative example, let's say we have the following. ***For the real raffle, these values must be replaced by those for the action first-prize redemption token.***

In [12]:
# Replace with actual policy and name!
PRIZE1_POLICY=8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
PRIZE1_NAME=1stPrize

Now we compute the Plutus datum that must be attached to the UTxO sent to the payout address. ***It is critically important that this datum be correct because the funds in the UTxO will be otherwise locked forever.*** Review carefully and consult if you have any doubts that the datum might be incorrect!

In [13]:
marlowe-cli role datum \
  --roles-currency $PRIZE1_POLICY \
  --role-name $PRIZE1_NAME \
  --out-file first-prize.datum

2adb03fbdb1f0302528255d895b491b283c866b5d1f8053a44e5c8c70f1e9dd8


At this point, send the first prize tokens to the sponsor address. ***The instructions below require that there be no other tokens other than the prize ones at the sponsor address.***

In [14]:
SPONSOR_SKEY=sponsor.skey
SPONSOR_ADDR=$(cat sponsor.mainnet.address)
echo "SPONSOR_ADDR = $SPONSOR_ADDR"

SPONSOR_ADDR = addr1q95e9feu4hkp4qwvgqasq02na05z3eg33zzjquf2d86e6qzznwng4gtlladnxm7d486psa003jy6dv230t82rvv3pflqeuzzt2


After the tokens have been sent, we can query to see them.

In [15]:
cardano-cli query utxo --mainnet --address $SPONSOR_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
d2f089f5bba04fc6fa561cdd056612eba28cc339037236a083647764678fcf8c     1        30000000 lovelace + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.000de1404f544b50697261746532343835 + 60000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.434e43416c61 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.436861696e734f665761723038323731 + 350000000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.456d706f7761 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4669727374436f756e63696c30373437 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4761796130383639 + 125000000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.484f534b59 + 125000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d495241 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c

Now build the transaction that moves the prize ada and tokens to a UTxO at the Marlowe role-payout address ***and with the correct datum***.

In [16]:
# Ignore this cell, which is just present because we are mocking up the first prize.
PRIZE1_VALUE="30000000+1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d6574657261426c75653236+125000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d495241 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d696c6573746f6e6531434e43416c6131353837 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d696c6573746f6e6532434e43416c6131363238 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d696c6573746f6e6533434e43416c6131323633 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d696c6573746f6e6534434e43416c6130393030 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d696c6573746f6e6536434e43416c6131353337 + 700000000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.50524f584945 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.50726f786965576f6d62 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.000de1404f544b50697261746532343835 + 60000000000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4e45574d + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4669727374436f756e63696c30373437 + 350000000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.456d706f7761 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4761796130383639 + 60000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.434e43416c61 + 350000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4f544b2052756d + 25000000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.484f534b59 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.436861696e734f665761723038323731+100000000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.484f534b59+1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d65636861537461673030393032+1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.636e632e616c612d726166666c6531+1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d696c6573746f6e6535434e43416c6131393537+1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d696c6573746f6e6537434e43416c6131353234"

In [17]:
cardano-cli query protocol-parameters \
  --mainnet \
  --out-file network.protocol
  
cardano-cli transaction build \
  --mainnet \
  --babbage-era \
  --protocol-params-file network.protocol \
  $(cardano-cli query utxo --mainnet --address $SPONSOR_ADDR | sed -e '1,2d' -e 's/^\([^ ]*\)  *\([^ ]*\) .*$/ --tx-in \1#\2/') \
  --tx-out "$PAYOUT_ADDR+$PRIZE1_VALUE" \
    --tx-out-datum-embed-file first-prize.datum \
  --change-address $SPONSOR_ADDR \
  --out-file first-prize.unsigned

Estimated transaction fee: Lovelace 195421


***For safety, review the transaction carefully. Ask for assistance if in doubt that the transaction is correct.***

In [18]:
cardano-cli transaction view --tx-body-file first-prize.unsigned

auxiliary scripts: null
certificates: null
collateral inputs: []
era: Babbage
fee: 195421 Lovelace
inputs:
- d2f089f5bba04fc6fa561cdd056612eba28cc339037236a083647764678fcf8c#1
- f5666a247fe337680660cf3e56d03e22146f7530ed8b2350597a634642516611#3
metadata: null
mint: null
outputs:
- address: addr1w8sk2cgzxg34hwa7ladenzer8k4wg2temmyj5eezm8x6nzg35nqj2
  address era: Shelley
  amount:
    lovelace: 30000000
    policy 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d:
      asset 000de1404f544b50697261746532343835: 1
      asset 434e43416c61 (CNCAla): 60000
      asset 436861696e734f665761723038323731 (ChainsOfWar08271): 1
      asset 456d706f7761 (Empowa): 350000000
      asset 4669727374436f756e63696c30373437 (FirstCouncil0747): 1
      asset 4761796130383639 (Gaya0869): 1
      asset 484f534b59 (HOSKY): 125000000
      asset 4d495241 (MIRA): 125000
      asset 4d65636861537461673030393032 (MechaStag00902): 1
      asset 4d6574657261426c75653236 (MeteraBlue26): 1
      asset 4d696c

Sign the transaction.

In [19]:
cardano-cli transaction sign \
  --mainnet \
  --signing-key-file $SPONSOR_SKEY \
  --tx-body-file first-prize.unsigned \
  --out-file first-prize.signed

Submit the transaction.

In [20]:
cardano-cli transaction submit \
  --mainnet \
  --tx-file first-prize.signed

Transaction successfully submitted.


Compute the transaction ID and view the result on an explorer.

In [21]:
PRIZE1_TX="$(cardano-cli transaction txid --tx-file first-prize.signed)"
PRIZE1_UTXO="$PRIZE1_TX#0"
echo "PRIZE1_UTXO = $PRIZE1_UTXO"
echo "https://cardanoscan.io/transaction/$PRIZE1_TX?tab=utxo"

PRIZE1_UTXO = 1363aba2fb038fe9fadc8a71268287707e4dec5c9344d50a36de3ebd312b9710#0
https://cardanoscan.io/transaction/1363aba2fb038fe9fadc8a71268287707e4dec5c9344d50a36de3ebd312b9710?tab=utxo


#### Second prize

Now repeat the same steps for the second prize.

For the purposes of this narrative example, let's say we have the following. For the real raffle, these values must be replaced by those for the action second-prize redemption token.

In [22]:
# Replace with actual policy and name!
PRIZE2_POLICY=8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
PRIZE2_NAME=2ndPrize

Now we compute the Plutus datum that must be attached to the UTxO sent to the payout address. ***It is critically important that this datum be correct because the funds in the UTxO will be otherwise locked forever.*** Review carefully and consult if you have any doubts that the datum might be incorrect!

In [23]:
marlowe-cli role datum \
  --roles-currency $PRIZE2_POLICY \
  --role-name $PRIZE2_NAME \
  --out-file second-prize.datum

481b24bb5503d1b180e2c83e3ca2116d178ee4e7bf63ac038063391f336b142b


At this point, send the second prize tokens to the sponsor address. ***The instructions below require that there be no other tokens other than the prize ones at the sponsor address.***

After the tokens have been sent, we can query to see them.

In [24]:
cardano-cli query utxo --mainnet --address $SPONSOR_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
1363aba2fb038fe9fadc8a71268287707e4dec5c9344d50a36de3ebd312b9710     1        9804579 lovelace + TxOutDatumNone
b9eb32f18ab2eabefac27d1d473127c6c72a21021dacb64e35559c95ce916918     1        25000000 lovelace + 35000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.434e43416c61 + 100000000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.456d706f7761 + 50000000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.484f534b59 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.48617264466f726b456173656c6c6530303234 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4a41524f44495254343835 + 50000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d495241 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d65636861537461673031343331 + 1 8bb3b343d8e404472337966a72215

Now build the transaction that moves the prize ada and tokens to a UTxO at the Marlowe role-payout address ***and with the correct datum***.

In [25]:
# Ignore this cell, which is just present because we are mocking up the second prize.
PRIZE2_VALUE="25000000+1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.636e632e616c612d726166666c6532+50000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d495241 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d696c6573746f6e6531434e43416c6130363239 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d696c6573746f6e6532434e43416c6131353338 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d696c6573746f6e6533434e43416c6130373839 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d696c6573746f6e6534434e43416c6130333535 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d696c6573746f6e6535434e43416c6131343835 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d696c6573746f6e6536434e43416c6131323732 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d696c6573746f6e6537434e43416c6131303033 + 200000000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.50524f584945 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.576973646f6d30313037 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4a41524f44495254343835 + 35000000000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4e45574d + 100000000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.456d706f7761 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.5361627265636174313234 + 35000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.434e43416c61 + 100000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4f544b2052756d + 50000000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.484f534b59 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.48617264466f726b456173656c6c6530303234 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d6574657261426c75653237+1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d65636861537461673031343331"

In [26]:
cardano-cli transaction build \
  --mainnet \
  --babbage-era \
  --protocol-params-file network.protocol \
  $(cardano-cli query utxo --mainnet --address $SPONSOR_ADDR | sed -e '1,2d' -e 's/^\([^ ]*\)  *\([^ ]*\) .*$/ --tx-in \1#\2/') \
  --tx-out "$PAYOUT_ADDR+$PRIZE2_VALUE" \
    --tx-out-datum-embed-file second-prize.datum \
  --change-address $SPONSOR_ADDR \
  --out-file second-prize.unsigned

Estimated transaction fee: Lovelace 194541


***For safety, review the transaction carefully. Ask for assistance if in doubt that the transaction is correct.***

In [27]:
cardano-cli transaction view --tx-body-file second-prize.unsigned

auxiliary scripts: null
certificates: null
collateral inputs: []
era: Babbage
fee: 194541 Lovelace
inputs:
- 1363aba2fb038fe9fadc8a71268287707e4dec5c9344d50a36de3ebd312b9710#1
- b9eb32f18ab2eabefac27d1d473127c6c72a21021dacb64e35559c95ce916918#1
metadata: null
mint: null
outputs:
- address: addr1w8sk2cgzxg34hwa7ladenzer8k4wg2temmyj5eezm8x6nzg35nqj2
  address era: Shelley
  amount:
    lovelace: 25000000
    policy 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d:
      asset 434e43416c61 (CNCAla): 35000
      asset 456d706f7761 (Empowa): 100000000
      asset 484f534b59 (HOSKY): 50000000
      asset 48617264466f726b456173656c6c6530303234 (HardForkEaselle0024): 1
      asset 4a41524f44495254343835 (JARODIRT485): 1
      asset 4d495241 (MIRA): 50000
      asset 4d65636861537461673031343331 (MechaStag01431): 1
      asset 4d6574657261426c75653237 (MeteraBlue27): 1
      asset 4d696c6573746f6e6531434e43416c6130363239 (Milestone1CNCAla0629): 1
      asset 4d696c6573746f6e6532434e4341

Sign the transaction.

In [28]:
cardano-cli transaction sign \
  --mainnet \
  --signing-key-file $SPONSOR_SKEY \
  --tx-body-file second-prize.unsigned \
  --out-file second-prize.signed

Submit the transaction.

In [29]:
cardano-cli transaction submit \
  --mainnet \
  --tx-file second-prize.signed

Transaction successfully submitted.


Compute the transaction ID and view the result on an explorer.

In [30]:
PRIZE2_TX="$(cardano-cli transaction txid --tx-file second-prize.signed)"
PRIZE2_UTXO="$PRIZE2_TX#0"
echo "PRIZE2_UTXO = $PRIZE2_UTXO"
echo "https://cardanoscan.io/transaction/$PRIZE2_TX?tab=utxo"

PRIZE2_UTXO = 286a21a9101d50f5cb9b6e7f80efbee8ff35ea347a0f7007ca09514a5b845e2c#0
https://cardanoscan.io/transaction/286a21a9101d50f5cb9b6e7f80efbee8ff35ea347a0f7007ca09514a5b845e2c?tab=utxo


#### Third prize

Now repeat the same steps for the third prize.

For the purposes of this narrative example, let's say we have the following. For the real raffle, these values must be replaced by those for the action third-prize redemption token.

In [31]:
# Replace with actual policy and name!
PRIZE3_POLICY=8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
PRIZE3_NAME=3rdPrize

Now we compute the Plutus datum that must be attached to the UTxO sent to the payout address. ***It is critically important that this datum be correct because the funds in the UTxO will be otherwise locked forever.*** Review carefully and consult if you have any doubts that the datum might be incorrect!

In [32]:
marlowe-cli role datum \
  --roles-currency $PRIZE3_POLICY \
  --role-name $PRIZE3_NAME \
  --out-file third-prize.datum

29648f212f65a4a84452ef2ea94c84dc8c251f9859f94937824f0d32a2aaaf3b


At this point, send the third prize tokens to the sponsor address. ***The instructions below require that there be no other tokens other than the prize ones at the sponsor address.***

After the tokens have been sent, we can query to see them.

In [33]:
cardano-cli query utxo --mainnet --address $SPONSOR_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
286a21a9101d50f5cb9b6e7f80efbee8ff35ea347a0f7007ca09514a5b845e2c     1        9610038 lovelace + TxOutDatumNone
eea361ac873f431428271f3e6b1208ef26c0e812911bdef3c309f788614e43cb     1        20000000 lovelace + 5000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.434e43416c61 + 50000000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.456d706f7761 + 25000000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.484f534b59 + 25000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d495241 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d65636861537461673032333331 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d6574657261426c75653239 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d696c6573746f6e6531434e43416c6130303634 + 1 8bb3b343d8e404472337966a722

Now build the transaction that moves the prize ada and tokens to a UTxO at the Marlowe role-payout address ***and with the correct datum***.

In [34]:
# Ignore this cell, which is just present because we are mocking up the third prize.
PRIZE3_VALUE="20000000+1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d65636861537461673032333331+100000000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.50494e4b50414e54484552+1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.636e632e616c612d726166666c6533+25000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d495241 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d696c6573746f6e6531434e43416c6130303634 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d696c6573746f6e6532434e43416c6131333239 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d696c6573746f6e6533434e43416c6130373233 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d696c6573746f6e6534434e43416c6130303434 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d696c6573746f6e6535434e43416c6131333730 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d696c6573746f6e6536434e43416c6130333635 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d696c6573746f6e6537434e43416c6130373033 + 100000000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.50524f584945 + 5000000000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4e45574d + 50000000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.456d706f7761 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4f544b46616d696c794372657374333230 + 5000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.434e43416c61 + 50000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4f544b2052756d + 25000000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.484f534b59 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d6574657261426c75653239 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.77686572657343617264616e6f50726f7869657357616c646f363739 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.576973646f6d4f7264696e616c7330333032"

In [35]:
cardano-cli transaction build \
  --mainnet \
  --babbage-era \
  --protocol-params-file network.protocol \
  $(cardano-cli query utxo --mainnet --address $SPONSOR_ADDR | sed -e '1,2d' -e 's/^\([^ ]*\)  *\([^ ]*\) .*$/ --tx-in \1#\2/') \
  --tx-out "$PAYOUT_ADDR+$PRIZE3_VALUE" \
    --tx-out-datum-embed-file third-prize.datum \
  --change-address $SPONSOR_ADDR \
  --out-file third-prize.unsigned

Estimated transaction fee: Lovelace 195685


***For safety, review the transaction carefully. Ask for assistance if in doubt that the transaction is correct.***

In [36]:
cardano-cli transaction view --tx-body-file third-prize.unsigned

auxiliary scripts: null
certificates: null
collateral inputs: []
era: Babbage
fee: 195685 Lovelace
inputs:
- 286a21a9101d50f5cb9b6e7f80efbee8ff35ea347a0f7007ca09514a5b845e2c#1
- eea361ac873f431428271f3e6b1208ef26c0e812911bdef3c309f788614e43cb#1
metadata: null
mint: null
outputs:
- address: addr1w8sk2cgzxg34hwa7ladenzer8k4wg2temmyj5eezm8x6nzg35nqj2
  address era: Shelley
  amount:
    lovelace: 20000000
    policy 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d:
      asset 434e43416c61 (CNCAla): 5000
      asset 456d706f7761 (Empowa): 50000000
      asset 484f534b59 (HOSKY): 25000000
      asset 4d495241 (MIRA): 25000
      asset 4d65636861537461673032333331 (MechaStag02331): 1
      asset 4d6574657261426c75653239 (MeteraBlue29): 1
      asset 4d696c6573746f6e6531434e43416c6130303634 (Milestone1CNCAla0064): 1
      asset 4d696c6573746f6e6532434e43416c6131333239 (Milestone2CNCAla1329): 1
      asset 4d696c6573746f6e6533434e43416c6130373233 (Milestone3CNCAla0723): 1
      asset 

Sign the transaction.

In [39]:
cardano-cli transaction sign \
  --mainnet \
  --signing-key-file $SPONSOR_SKEY \
  --tx-body-file third-prize.unsigned \
  --out-file third-prize.signed

Submit the transaction.

In [40]:
cardano-cli transaction submit \
  --mainnet \
  --tx-file third-prize.signed

Transaction successfully submitted.


Compute the transaction ID and view the result on an explorer.

In [41]:
PRIZE3_TX="$(cardano-cli transaction txid --tx-file third-prize.signed)"
PRIZE3_UTXO="$PRIZE3_TX#0"
echo "PRIZE3_UTXO = $PRIZE3_UTXO"
echo "https://cardanoscan.io/transaction/$PRIZE3_TX?tab=utxo"

PRIZE3_UTXO = 21a7c06555f75bb869d09a503f94de5f57755165e3c7f38b9ef8a1ad86065e60#0
https://cardanoscan.io/transaction/21a7c06555f75bb869d09a503f94de5f57755165e3c7f38b9ef8a1ad86065e60?tab=utxo


#### Recap

Now we have bundled the three prizes into three UTxOs at the Marlowe role-payout address. Each is associated with the Plutus datum that lets the holder of the first, second, or third prize token redeem them, respectively.

In [42]:
echo "PRIZE1_UTXO = $PRIZE1_UTXO"
cardano-cli query utxo --mainnet --tx-in $PRIZE1_UTXO
echo "https://cardanoscan.io/transaction/$PRIZE1_TX?tab=utxo"

PRIZE1_UTXO = 1363aba2fb038fe9fadc8a71268287707e4dec5c9344d50a36de3ebd312b9710#0
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
1363aba2fb038fe9fadc8a71268287707e4dec5c9344d50a36de3ebd312b9710     0        30000000 lovelace + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.000de1404f544b50697261746532343835 + 60000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.434e43416c61 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.436861696e734f665761723038323731 + 350000000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.456d706f7761 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4669727374436f756e63696c30373437 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4761796130383639 + 125000000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.484f534b59 + 125000 8bb3b343d8e404472337966a722150048c768d0a

In [43]:
echo "PRIZE2_UTXO = $PRIZE2_UTXO"
cardano-cli query utxo --mainnet --tx-in $PRIZE2_UTXO
echo "https://cardanoscan.io/transaction/$PRIZE2_TX?tab=utxo"

PRIZE2_UTXO = 286a21a9101d50f5cb9b6e7f80efbee8ff35ea347a0f7007ca09514a5b845e2c#0
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
286a21a9101d50f5cb9b6e7f80efbee8ff35ea347a0f7007ca09514a5b845e2c     0        25000000 lovelace + 35000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.434e43416c61 + 100000000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.456d706f7761 + 50000000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.484f534b59 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.48617264466f726b456173656c6c6530303234 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4a41524f44495254343835 + 50000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d495241 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d65636861537461673031343331 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d6

In [44]:
echo "PRIZE3_UTXO = $PRIZE3_UTXO"
cardano-cli query utxo --mainnet --tx-in $PRIZE3_UTXO
echo "https://cardanoscan.io/transaction/$PRIZE3_TX?tab=utxo"

PRIZE3_UTXO = 21a7c06555f75bb869d09a503f94de5f57755165e3c7f38b9ef8a1ad86065e60#0
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
21a7c06555f75bb869d09a503f94de5f57755165e3c7f38b9ef8a1ad86065e60     0        20000000 lovelace + 5000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.434e43416c61 + 50000000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.456d706f7761 + 25000000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.484f534b59 + 25000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d495241 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d65636861537461673032333331 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d6574657261426c75653239 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d696c6573746f6e6531434e43416c6130303634 + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4

## Running the raffle

### Ensure that the three prize tokens reside at the sponsor address

Send the three prize tokens (the Marlowe role tokens) to the sponsor address.

In [45]:
SPONSOR_SKEY=sponsor.skey
SPONSOR_ADDR=$(cat sponsor.mainnet.address)
echo "SPONSOR_ADDR = $SPONSOR_ADDR"

SPONSOR_ADDR = addr1q95e9feu4hkp4qwvgqasq02na05z3eg33zzjquf2d86e6qzznwng4gtlladnxm7d486psa003jy6dv230t82rvv3pflqeuzzt2


After the tokens have been sent, we can query to see them.

In [46]:
cardano-cli query utxo --mainnet --address $SPONSOR_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
21a7c06555f75bb869d09a503f94de5f57755165e3c7f38b9ef8a1ad86065e60     1        9414353 lovelace + TxOutDatumNone
29b464c1f8cc56d6a068b03147c0f5acf26ce023d68623d1e81a0a4e542c4851     0        50000000 lovelace + TxOutDatumNone
a117aba95e03ec263fd090161431a97295f9fe95b504f10ad7058fe370656b65     1        1163700 lovelace + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.3372645072697a65 + TxOutDatumNone
b082402ffa76bd652ff215c69801258612f06b97ff6cfa3b734792b0340a274b     1        1163700 lovelace + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.326e645072697a65 + TxOutDatumNone
c870610fd99dd9b7e7a6fb98ade9a69579d990923503a81a3943af0fff9e603d     1        1163700 lovelace + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.3173745072697a65 + TxOutDatumNone


### Download the executables to run the raffle.

The script residing at https://github.com/input-output-hk/real-world-marlowe/tree/main/archives/raffle are compiled to executables that should work on any recent Linux machine.

In [47]:
curl -sS http://dl.marlowestat.org/raffle-bin.tar -o raffle-bin.tar

Unpack the executable files.

In [48]:
tar xvf raffle-bin.tar

bin/
bin/cardano-cli
bin/marlowe-cli
bin/marlowe-runtime-cli
bin/InitializeRaffle
bin/ExecuteRaffle


### Run the raffle for the first prize

First create the file that describes the role token holding the first prize.

In [49]:
cat << EOI > first-prize.token
[
    [
        "$PRIZE1_POLICY",
        "$PRIZE1_NAME"
    ]
]
EOI
cat first-prize.token

[
    [
        "8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d",
        "1stPrize"
    ]
]


Now create the file with the addresses of the raffle ticket holders.  ***Note that the format of this file will change to include the ticket name and count in addition to the address.***

In [50]:
head first-prize.addresses

[
  "addr1q97q4e9nlhq08mrugs8pm5uq045e425w72mxhxupq9qqgykr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jq3dk3ng"
, "addr1qxmph7666hj5yadja34pdlm8n5un0fhc9p0hv2g9ask7m49n3kr657fpa3q8mzwmjqvl9lqdn9g2pm3ejhlgwpprwy2sq5s22j"
, "addr1q8pdjzhjthflmtu947feruy6rnu0zr8x7xlppgqxrew62xqwpjujxwjglw2gelk9thqcta802lprejcu4y8dtz64uhnsaeaet6"
, "addr1q9zun0s6thsyf9qt0zyfnn50x8kpwutrv20eq8dgx9gt6tkr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jq2x0dua"
, "addr1qy5sj3v265ucpuy7ea7h7qy533xnhcl9ru3g24xcetrnxu4n3kr657fpa3q8mzwmjqvl9lqdn9g2pm3ejhlgwpprwy2slm3vfx"
, "addr1qy5lsaapah8d5pf8s2uvnnas4yk9s55vgns4jsvzz5exksgwpjujxwjglw2gelk9thqcta802lprejcu4y8dtz64uhnsluyccl"
, "addr1qypet7s339suufnxmr4vrg09469v30g59yxa70yrkxvrfekr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqjg9sm2"
, "addr1qx26pe2wlf9x532vsh2cd6swfae2eanadz8kpqc7trzpfzdn3kr657fpa3q8mzwmjqvl9lqdn9g2pm3ejhlgwpprwy2sjgremk"
, "addr1q9jqjkxx0rjp08zffmwxyc3gtvkry56akn7p07dqjc649tgwpjujxwjglw2gelk9thqcta802lprejcu4y8dtz64uhns0qafe4"


Now create the configuration file for running the raffle.

In [51]:
yaml2json << EOI > first-prize.config
chunkSize: 5          # Danger: do not change the chunk size.
deadlines:
  deposit: 10d        # Deadline for depositing the prize token.
  selectWinner: 10d   # Deadline for selecting the winner.
  payout: 10d         # Deadline for paying the winner.
runtimeURI:
  host: 192.168.0.12  # The IP address of Marlowe Runtime v0.0.4.
  proxy_port: 53700   # The port number for Marlowe Runtime's proxy service.
  web_port: 53780     # The port number for Marlowe Runtime's web server.
sponsorAddressFilePath: sponsor.mainnet.address
sponsorCollateralFilePath: not-used
sponsorPrivateKeyFilePath: sponsor.skey
contract: first-prize.contract
state: first-prize.state
tmpTxToSign: temp.unsigned
tmpTxToSubmit: temp.signed
EOI
cat first-prize.config

{"chunkSize":5,"contract":"first-prize.contract","deadlines":{"deposit":"10d","payout":"10d","selectWinner":"10d"},"runtimeURI":{"host":"192.168.0.12","proxy_port":53700,"web_port":53780},"sponsorAddressFilePath":"sponsor.mainnet.address","sponsorCollateralFilePath":"not-used","sponsorPrivateKeyFilePath":"sponsor.skey","state":"first-prize.state","tmpTxToSign":"temp.unsigned","tmpTxToSubmit":"temp.signed"}


Now build and submit the transaction that initializes the raffle.

In [52]:
./bin/InitializeRaffle first-prize.config first-prize.addresses first-prize.token 2>/dev/null

{"blockHeaderHash":"b2ebd54ea0132ddb4239bc92ab3ed06d5b5d12e26e8786314b349cd5b94efdca","blockNo":9144321,"slotNo":100153169}
{"contractId":"127a11e2c5e4ce72cff5ab1462095747881b385bd22ff1b2a7996986e2d00428#1"}


Record the contract ID that was printed.

In [53]:
PRIZE1_CONTRACT=127a11e2c5e4ce72cff5ab1462095747881b385bd22ff1b2a7996986e2d00428#1

Run the raffle itself, which consists of several transactions:
1. Deposit the prize token.
2. Have the oracle retrieve a random number from https://www.random.org/.
3. Execute several Marlowe notify transactions to locate the winner.
4. Send the prize token to the winner.

***Note that the output below will be changed to also print the address of the winner and the name of the raffle ticket that they held.***

In [54]:
./bin/ExecuteRaffle first-prize.config first-prize.addresses first-prize.token $PRIZE1_CONTRACT 2>/dev/null


#########################
WaitingForPrizeDeposit
 >> Depositing 1stPrize
{"txId":"22c6194301bd8f6d858138116113fe8c632b96768548a6191fe72681bf8ae62a"}
{"blockHeaderHash":"462d5409c5df1222f99d9eb969794f7f6f1a19014a0c9996fd40ea1c22efdb89","blockNo":9144328,"slotNo":100153422}
 >> 1stPrize deposited
>>>> Prize NFTs Deposited
#########################

#########################
WaitingForOracle
#########################
{"txId":"1d7aa8b42031f3dbfb30b320f437010419ea6a46409a9b39ff84ad8a5c2898d9"}
{"blockHeaderHash":"4993d6034b6fbd8b0380f31377ae9d89db92de47532ae187aca2ad38310de9ea","blockNo":9144329,"slotNo":100153432}
>>>> Oracle has answered with 581
#########################

#########################
WaitingForNotify
#########################
{"txId":"40453b69c70d1fd4046c5af29ec840f410b5a5ceead54f2bc02e5e7c49de2270"}
{"blockHeaderHash":"c0f516ac74f82b802141f29f61aba690877682929be6ec45ee856a3312ba7526","blockNo":9144330,"slotNo":100153444}
 >> Notified
#########################

###########

View the contract on MarloweScan.

In [55]:
echo "https://mainnet.marlowescan.com/contractView?tab=tx-list&contractId=${PRIZE1_CONTRACT%%#1}%231"

https://mainnet.marlowescan.com/contractView?tab=tx-list&contractId=127a11e2c5e4ce72cff5ab1462095747881b385bd22ff1b2a7996986e2d00428%231


Record the random number from that the oracle choose, listed in the output above.

In [56]:
PRIZE1_CHOICE=581

Compute the address of the winner and view their prize on an explorer.

In [57]:
PRIZE1_WINNER=$(jq -r ".[$PRIZE1_CHOICE]" first-prize.addresses)
echo "PRIZE1_WINNER = $PRIZE1_WINNER"
echo "https://cardanoscan.io/address/addr1qyhayn5lzjyncmrf55tw09wur765numtexqwrx95jeuwfccwpjujxwjglw2gelk9thqcta802lprejcu4y8dtz64uhnsksgpug"

PRIZE1_WINNER = addr1qyhayn5lzjyncmrf55tw09wur765numtexqwrx95jeuwfccwpjujxwjglw2gelk9thqcta802lprejcu4y8dtz64uhnsksgpug
https://cardanoscan.io/address/addr1qyhayn5lzjyncmrf55tw09wur765numtexqwrx95jeuwfccwpjujxwjglw2gelk9thqcta802lprejcu4y8dtz64uhnsksgpug


### Run the raffle for the second prize

First create the file that describes the role token holding the second prize.

In [58]:
cat << EOI > second-prize.token
[
    [
        "$PRIZE2_POLICY",
        "$PRIZE2_NAME"
    ]
]
EOI
cat second-prize.token

[
    [
        "8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d",
        "2ndPrize"
    ]
]


Now create the file for those eligible for the second prize by deleting the entry for the first prize.

In [59]:
jq "del(.[$PRIZE1_CHOICE])" first-prize.addresses > second-prize.addresses

Now create the configuration file for running the raffle.

In [60]:
yaml2json << EOI > second-prize.config
chunkSize: 5          # Danger: do not change the chunk size.
deadlines:
  deposit: 10d        # Deadline for depositing the prize token.
  selectWinner: 10d   # Deadline for selecting the winner.
  payout: 10d         # Deadline for paying the winner.
runtimeURI:
  host: 192.168.0.12  # The IP address of Marlowe Runtime v0.0.4.
  proxy_port: 53700   # The port number for Marlowe Runtime's proxy service.
  web_port: 53780     # The port number for Marlowe Runtime's web server.
sponsorAddressFilePath: sponsor.mainnet.address
sponsorCollateralFilePath: not-used
sponsorPrivateKeyFilePath: sponsor.skey
contract: second-prize.contract
state: second-prize.state
tmpTxToSign: temp.unsigned
tmpTxToSubmit: temp.signed
EOI
cat second-prize.config

{"chunkSize":5,"contract":"second-prize.contract","deadlines":{"deposit":"10d","payout":"10d","selectWinner":"10d"},"runtimeURI":{"host":"192.168.0.12","proxy_port":53700,"web_port":53780},"sponsorAddressFilePath":"sponsor.mainnet.address","sponsorCollateralFilePath":"not-used","sponsorPrivateKeyFilePath":"sponsor.skey","state":"second-prize.state","tmpTxToSign":"temp.unsigned","tmpTxToSubmit":"temp.signed"}


Now build and submit the transaction that initializes the raffle.

In [61]:
./bin/InitializeRaffle second-prize.config second-prize.addresses second-prize.token 2>/dev/null

{"blockHeaderHash":"54dd869f501922eef8bf110a2793e9045e6c26f72f202d74f0a675aa20a78696","blockNo":9144362,"slotNo":100154325}
{"contractId":"f0a36b7d4e82a8bcdffb4ffac3751e8ea27757e05d9bf28841f7a0cc463988c7#1"}


Record the contract ID that was printed.

In [62]:
PRIZE2_CONTRACT=f0a36b7d4e82a8bcdffb4ffac3751e8ea27757e05d9bf28841f7a0cc463988c7#1

Run the raffle itself, which consists of several transactions.

***Note that the output below will be changed to also print the address of the winner and the name of the raffle ticket that they held.***

In [63]:
./bin/ExecuteRaffle second-prize.config second-prize.addresses second-prize.token $PRIZE2_CONTRACT 2>/dev/null


#########################
WaitingForPrizeDeposit
 >> Depositing 2ndPrize
{"txId":"c7144608160b87be4b1dac93e9e2aae9a7752d4cf402898173fb67fff64567f6"}
{"blockHeaderHash":"26e779f3c10a80991758da74871e82e4502314c7bae86a15750056c05d379a91","blockNo":9144367,"slotNo":100154390}
 >> 2ndPrize deposited
>>>> Prize NFTs Deposited
#########################

#########################
WaitingForOracle
#########################
{"txId":"59e5e6ffecef373d28db36fe427c84bdbce8d22ec962309efe95807c668a8774"}
{"blockHeaderHash":"b78a9834451cdfeb256637505f6f04d812eb681548b0b5fbd18d8a87e6f32149","blockNo":9144369,"slotNo":100154440}
>>>> Oracle has answered with 397
#########################

#########################
WaitingForNotify
#########################
{"txId":"682af50c6e6207c5350afbfea7f71e7299da0dcea7498cd41a1488fb627b0622"}
{"blockHeaderHash":"08dbd44752a565902b792e74b3ed19d27c70391c4d2a154d1ca18e049cccfc6c","blockNo":9144370,"slotNo":100154450}
 >> Notified
#########################

###########

View the contract on MarloweScan.

In [64]:
echo "https://mainnet.marlowescan.com/contractView?tab=tx-list&contractId=${PRIZE2_CONTRACT%%#1}%231"

https://mainnet.marlowescan.com/contractView?tab=tx-list&contractId=f0a36b7d4e82a8bcdffb4ffac3751e8ea27757e05d9bf28841f7a0cc463988c7%231


Record the random number from that the oracle choose, listed in the output above.

In [65]:
PRIZE2_CHOICE=397

Compute the address of the winner and view their prize on an explorer.

In [66]:
PRIZE2_WINNER=$(jq -r ".[$PRIZE2_CHOICE]" second-prize.addresses)
echo "PRIZE2_WINNER = $PRIZE2_WINNER"
echo "https://cardanoscan.io/address/$PRIZE2_WINNER"

PRIZE2_WINNER = addr1q90cwxeuz89l0lfaw2fhjjstcj7ypxakd5v65rls0d7wkndn3kr657fpa3q8mzwmjqvl9lqdn9g2pm3ejhlgwpprwy2sdrqmkg
https://cardanoscan.io/address/addr1q90cwxeuz89l0lfaw2fhjjstcj7ypxakd5v65rls0d7wkndn3kr657fpa3q8mzwmjqvl9lqdn9g2pm3ejhlgwpprwy2sdrqmkg


### Run the raffle for the thrid prize

First create the file that describes the role token holding the third prize.

In [67]:
cat << EOI > third-prize.token
[
    [
        "$PRIZE3_POLICY",
        "$PRIZE3_NAME"
    ]
]
EOI
cat third-prize.token

[
    [
        "8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d",
        "3rdPrize"
    ]
]


Now create the file for those eligible for the second prize by deleting the entry for the second prize.

In [68]:
jq "del(.[$PRIZE2_CHOICE])" second-prize.addresses > third-prize.addresses

Now create the configuration file for running the raffle.

In [69]:
yaml2json << EOI > third-prize.config
chunkSize: 5          # Danger: do not change the chunk size.
deadlines:
  deposit: 10d        # Deadline for depositing the prize token.
  selectWinner: 10d   # Deadline for selecting the winner.
  payout: 10d         # Deadline for paying the winner.
runtimeURI:
  host: 192.168.0.12  # The IP address of Marlowe Runtime v0.0.4.
  proxy_port: 53700   # The port number for Marlowe Runtime's proxy service.
  web_port: 53780     # The port number for Marlowe Runtime's web server.
sponsorAddressFilePath: sponsor.mainnet.address
sponsorCollateralFilePath: not-used
sponsorPrivateKeyFilePath: sponsor.skey
contract: third-prize.contract
state: third-prize.state
tmpTxToSign: temp.unsigned
tmpTxToSubmit: temp.signed
EOI
cat third-prize.config

{"chunkSize":5,"contract":"third-prize.contract","deadlines":{"deposit":"10d","payout":"10d","selectWinner":"10d"},"runtimeURI":{"host":"192.168.0.12","proxy_port":53700,"web_port":53780},"sponsorAddressFilePath":"sponsor.mainnet.address","sponsorCollateralFilePath":"not-used","sponsorPrivateKeyFilePath":"sponsor.skey","state":"third-prize.state","tmpTxToSign":"temp.unsigned","tmpTxToSubmit":"temp.signed"}


Now build and submit the transaction that initializes the raffle.

In [70]:
./bin/InitializeRaffle third-prize.config third-prize.addresses third-prize.token 2>/dev/null

{"blockHeaderHash":"2a7b19e317f44fe384cb8ae75c583ede4757da0bddbc4258cdc91f458ade5d50","blockNo":9144386,"slotNo":100154692}
{"contractId":"a1e0b1ff9743fc501f5e07f8db1ef9b148ea9e31f1ea190b1afaebc59ec0faac#1"}


Record the contract ID that was printed.

In [71]:
PRIZE3_CONTRACT=a1e0b1ff9743fc501f5e07f8db1ef9b148ea9e31f1ea190b1afaebc59ec0faac#1

Run the raffle itself, which consists of several transactions.

***Note that the output below will be changed to also print the address of the winner and the name of the raffle ticket that they held.***

In [72]:
./bin/ExecuteRaffle third-prize.config third-prize.addresses third-prize.token $PRIZE3_CONTRACT 2>/dev/null


#########################
WaitingForPrizeDeposit
 >> Depositing 3rdPrize
{"txId":"5a9831f7f25b92b958982bea1d2fe462b2ae0d4ef9beb7e823262a496f03ef32"}
{"blockHeaderHash":"f883ca737ee17767ed717ccbafefe45cebe67f5d9b7827ebeb17de3be9e687b5","blockNo":9144388,"slotNo":100154744}
 >> 3rdPrize deposited
>>>> Prize NFTs Deposited
#########################

#########################
WaitingForOracle
#########################
{"txId":"e53b5ca51af323bb34817c2d45e232605f05237fb73ba4b5b5812a7c3367117d"}
{"blockHeaderHash":"4706eb926cd487a95d4da4ddf10767482cdd660f8bcbb99c0d88b2bdda330951","blockNo":9144390,"slotNo":100154766}
>>>> Oracle has answered with 205
#########################

#########################
WaitingForNotify
#########################
{"txId":"951d92a213ace6e65f67029022a2452c40d9808320971f80e1ab95dbab5dac46"}
{"blockHeaderHash":"75ae929eaeb9dab2b481a628a26968a55bfe87163c22e273342aad3b4d0f79f7","blockNo":9144391,"slotNo":100154796}
 >> Notified
#########################

###########

View the contract on MarloweScan.

In [73]:
echo "https://mainnet.marlowescan.com/contractView?tab=tx-list&contractId=${PRIZE3_CONTRACT%%#1}%231"

https://mainnet.marlowescan.com/contractView?tab=tx-list&contractId=a1e0b1ff9743fc501f5e07f8db1ef9b148ea9e31f1ea190b1afaebc59ec0faac%231


Record the random number from that the oracle choose, listed in the output above.

In [74]:
PRIZE3_CHOICE=205

Compute the address of the winner and view their prize on an explorer.

In [75]:
PRIZE3_WINNER=$(jq -r ".[$PRIZE3_CHOICE]" third-prize.addresses)
echo "PRIZE3_WINNER = $PRIZE_WINNER"
echo "https://cardanoscan.io/address/$PRIZE3_WINNER"

PRIZE3_WINNER = 
https://cardanoscan.io/address/addr1q8yegd5a0tcxrf7xv9c82ft4qpnd5yyrv867v7ca55auzp4n3kr657fpa3q8mzwmjqvl9lqdn9g2pm3ejhlgwpprwy2sc8h882
